In [127]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import ElasticNet, SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
df = pd.read_excel("ENB2012_data.xlsx")

In [215]:
%%file load_data.py
import pandas as pd
from sklearn.model_selection import train_test_split
def load_and_split_data(file_path, test_size=0.2, seed=22793):
    df = pd.read_excel(file_path)
    train_cols = list(df.columns)[:-2]
    test_cols = list(df.columns)[-2:]
    X = df[train_cols]
    Y1 = df[test_cols[0]]
    Y2 = df[test_cols[1]]
    X_train, X_test, Y1_train, Y1_test, Y2_train, Y2_test = train_test_split(X, Y1, Y2, test_size=test_size, random_state=seed)
    return X_train, X_test, Y1_train, Y1_test, Y2_train, Y2_test

Overwriting load_data.py


In [95]:
df.describe()

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

display(calc_vif(df.drop(['Y1', 'Y2', 'X2', 'X1', 'X3', 'X6'], axis=1)))

df.drop(['Y1', 'Y2'], axis=1).corr()
np.linalg.svd(df.drop(['Y1', 'Y2'], axis=1))

,variables,VIF
0,X4,5.245242
1,X5,4.544944
2,X7,4.284222
3,X8,4.485473


(array([[-0.02814056,  0.03509062, -0.03787135, ..., -0.0179358 ,
         -0.01754167, -0.01714755],
        [-0.02814077,  0.0350928 , -0.03314257, ...,  0.06183368,
          0.06120376,  0.06057384],
        [-0.02814098,  0.03509499, -0.0284138 , ...,  0.02969101,
          0.02911302,  0.02853503],
        ...,
        [-0.0429057 , -0.00809657,  0.00872139, ...,  0.99127378,
         -0.00804719, -0.00736817],
        [-0.04290592, -0.00809438,  0.01345016, ..., -0.00828407,
          0.99137322, -0.00896949],
        [-0.04290613, -0.00809219,  0.01817894, ..., -0.00784192,
         -0.00920637,  0.98942918]]),
 array([2.13257252e+04, 1.61355399e+03, 4.54048419e+01, 3.75007583e+01,
        2.84645236e+01, 3.60837091e+00, 1.63371309e+00, 1.73425532e-12]),
 array([[-9.70036530e-04, -8.80159198e-01, -4.13191831e-01,
         -2.33483684e-01, -6.56255926e-03, -4.51623918e-03,
         -3.02426371e-04, -3.62912786e-03],
        [ 1.85140796e-03, -2.42142773e-01,  8.13476566e-01,
   

In [12]:
# Check for any nulls in the data
for col in df.columns:
    display(df[df[col].isnull()])

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2


### As a baseline, two linear regression models will be trained independently on each target

In [103]:
seed = 22793

In [195]:
train_cols = list(df.columns)[:-2]
# train_cols = ['X4', 'X5', 'X6', 'X7']
test_cols = list(df.columns)[-2:]
X = df[train_cols]
Y1 = df[test_cols[0]]
Y2 = df[test_cols[1]]
X_train, X_test, Y1_train, Y1_test, Y2_train, Y2_test = train_test_split(X, Y1, Y2, test_size=0.2, random_state=seed)

kfolds = 5
scaler = StandardScaler()
scoring = 'neg_mean_squared_error'
# data is already centered, do not need intercept term
model_1 = make_pipeline(StandardScaler(), SGDRegressor(max_iter=10000))
model_2 = make_pipeline(StandardScaler(), SGDRegressor(max_iter=10000))
param_grid={'sgdregressor__alpha': np.arange(0.0, .015, .002)}
grid_search_1 = GridSearchCV(model_1, param_grid, scoring=scoring)
grid_search_2 = GridSearchCV(model_2, param_grid, scoring=scoring)

grid_search_1.fit(X_train, Y1_train, )
grid_search_2.fit(X_train, Y1_train, )

GridSearchCV(estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('sgdregressor',
                                        SGDRegressor(max_iter=10000))]),
             param_grid={'sgdregressor__alpha': array([0.   , 0.002, 0.004, 0.006, 0.008, 0.01 , 0.012, 0.014])},
             scoring='neg_mean_squared_error')

In [196]:
grid_search_1.cv_results_
print(grid_search_1.best_params_)
print(grid_search_2.best_params_)

{'sgdregressor__alpha': 0.0}
{'sgdregressor__alpha': 0.0}


In [197]:
total_mse = grid_search_1.best_score_ + grid_search_2.best_score_
total_mse = total_mse / 2
total_mse

-9.345767825387528

In [202]:
# Fit best model on entire training set and calculate test error
model_1 = make_pipeline(StandardScaler(), SGDRegressor(alpha=0, max_iter=10000))
model_2 = make_pipeline(StandardScaler(), SGDRegressor(alpha=0, max_iter=10000))
model_1 = model_1.fit(X_train, Y1_train)
model_2 = model_2.fit(X_train, Y2_train)


In [209]:
model_1_mse = mean_squared_error(model_1.predict(X_test), Y1_test)
model_2_mse = mean_squared_error(model_2.predict(X_test), Y2_test)
total_mse = np.mean([model_1_mse, model_2_mse])

In [210]:
total_mse

7.561500025535127